# 10 Minutes to cuDF's pandas accelerator mode (cudf.pandas)

cuDF is a Python GPU DataFrame library (built on the Apache Arrow columnar memory format) for loading, joining, aggregating, filtering, and otherwise manipulating tabular data using a DataFrame style API in the style of pandas.

As of the v23.10 release, cuDF now provides a pandas accelerator mode (`cudf.pandas`), allowing you to bring accelerated computing to your pandas workflows without requiring any code change.

This notebook is a short introduction to `cudf.pandas`.

In [0]:
!pip install --upgrade pip --extra-index-url=https://pypi.nvidia.com cudf-cu11 dask-cudf-cu11

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.4/506.4 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 71.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 12.0.1
    Not uninstalling pyarrow at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-fe134494-0b5a-4307-9fa4-ae9487b4f944
    Can't uninstall 'pyarrow'. No files were found to uninstall.
  Attempting uninstall: pip
    Found existin

# ⚠️ Verify your setup

First, we'll verify that you are running with an NVIDIA GPU and that cuDF is available.

If you haven't installed cuDF, please visit https://rapids.ai/#quick-start to choose your favorite installation method.

In [0]:
!pip list

Package                      Version
---------------------------- -------------
absl-py                      1.4.0
asttokens                    2.2.1
astunparse                   1.6.3
backcall                     0.2.0
blinker                      1.4
bokeh                        3.2.2
cachetools                   5.3.1
certifi                      2023.7.22
charset-normalizer           3.2.0
click                        8.1.7
cloudpickle                  3.0.0
comm                         0.1.3
contourpy                    1.1.0
cryptography                 3.4.8
cubinlinker-cu11             0.3.0.post1
cuda-python                  11.8.3
cudf-cu11                    23.12.1
cuml-cu11                    23.10.0
cupy-cuda11x                 12.3.0
cycler                       0.11.0
dask                         2023.11.0
dask-cuda                    23.10.0
dask-cudf-cu11               23.12.0
dask-databricks              0.3.0
dask-deltatable              0.3.1
dask-glm              

In [0]:
!nvidia-smi  # this should display information about available GPUs

Wed Jan 17 11:10:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:1E.0 Off |                    0 |
| N/A   19C    P8              11W /  70W |      2MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [0]:
import cudf  # this should work without any errors

We'll also install `plotly-express` for visualizing data.

In [0]:
#!pip install plotly_express

# Download the data

The data we'll be working with is the [Parking Violations Issued - Fiscal Year 2022](https://data.cityofnewyork.us/City-Government/Parking-Violations-Issued-Fiscal-Year-2022/7mxj-7a6y) dataset from NYC Open Data.

We're downloading a copy of this dataset from an s3 bucket hosted by NVIDIA to provide faster download speeds. We'll start by downloading the data. This should take about 30 seconds.

## Data License and Terms
As this dataset originates from the NYC Open Data Portal, it's governed by their license and terms of use.

### Are there restrictions on how I can use Open Data?

> Open Data belongs to all New Yorkers. There are no restrictions on the use of Open Data. Refer to Terms of Use for more information.

### [Terms of Use](https://opendata.cityofnewyork.us/overview/#termsofuse)

> By accessing datasets and feeds available through NYC Open Data, the user agrees to all of the Terms of Use of NYC.gov as well as the Privacy Policy for NYC.gov. The user also agrees to any additional terms of use defined by the agencies, bureaus, and offices providing data. Public data sets made available on NYC Open Data are provided for informational purposes. The City does not warranty the completeness, accuracy, content, or fitness for any particular purpose or use of any public data set made available on NYC Open Data, nor are any such warranties to be implied or inferred with respect to the public data sets furnished therein.

> The City is not liable for any deficiencies in the completeness, accuracy, content, or fitness for any particular purpose or use of any public data set, or application utilizing such data set, provided by any third party.

> Submitting City Agencies are the authoritative source of data available on NYC Open Data. These entities are responsible for data quality and retain version control of data sets and feeds accessed on the Site. Data may be updated, corrected, or refreshed at any time.

In [0]:
!wget https://data.rapids.ai/datasets/nyc_parking/nyc_parking_violations_2022.parquet

--2024-01-17 11:10:53--  https://data.rapids.ai/datasets/nyc_parking/nyc_parking_violations_2022.parquet
Resolving data.rapids.ai (data.rapids.ai)... 204.246.191.16, 204.246.191.112, 204.246.191.75, ...
Connecting to data.rapids.ai (data.rapids.ai)|204.246.191.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 477161608 (455M) [binary/octet-stream]
Saving to: ‘nyc_parking_violations_2022.parquet’

nyc_parking_violati 100%[===================>] 455.06M  33.8MB/s    in 16s     

2024-01-17 11:11:10 (29.2 MB/s) - ‘nyc_parking_violations_2022.parquet’ saved [477161608/477161608]



# Analysis using Standard Pandas

First, let's use Pandas to read in some columns of the dataset:

In [0]:
import pandas as pd

In [0]:
# read 5 columns data:
df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=[
        "Registration State",
        "Violation Description",
        "Vehicle Body Type",
        "Issue Date",
        "Summons Number",
    ],
)

# view a random sample of 10 rows:
df.sample(10)

,Registration State,Violation Description,Vehicle Body Type,Issue Date,Summons Number
3620710,NY,20-No Parking (Com Plate),VAN,09/22/2021,8858832280
9938779,TX,40-Fire Hydrant,2DSD,02/14/2022,8962643777
13201721,NY,PHTO SCHOOL ZN SPEED VIOLATION,2DSD,04/27/2022,4777188838
6096049,NY,FAILURE TO STOP AT RED LIGHT,VAN,10/26/2021,5117876726
785851,NY,19-No Stand (bus stop),VAN,07/16/2021,8951315381
8298974,NJ,BUS LANE VIOLATION,PV,12/21/2021,4021727000
13054096,NY,<NA>,SDN,05/13/2022,1471696534
11204869,IN,69-Fail to Dsp Prking Mtr Rcpt,TRAC,03/16/2022,8914800350
8308907,NY,BUS LANE VIOLATION,SUBN,12/28/2021,4021831794
10603176,NY,PHTO SCHOOL ZN SPEED VIOLATION,SUBN,03/16/2022,4771579751


Next, we'll try to answer a few questions using the data.

## Which parking violation is most commonly committed by vehicles from various U.S states?

Each record in our dataset contains the state of registration of the offending vehicle, and the type of parking offence. Let's say we want to get the most common type of offence for vehicles registered in different states. We can do this in Pandas using a combination of [value_counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) and [GroupBy.head](https://pandas.pydata.org/docs/reference/api/pandas.core.groupby.DataFrameGroupBy.head.html):

In [0]:
(
    df[["Registration State", "Violation Description"]]  # get only these two columns
    .value_counts()  # get the count of offences per state and per type of offence
    .groupby("Registration State")  # group by state
    .head(
        1
    )  # get the first row in each group (the type of offence with the largest count)
    .sort_index()  # sort by state name
    .reset_index()
)

,Registration State,Violation Description,0
0,99,74-Missing Display Plate,835
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
60,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
61,WA,21-No Parking (street clean),3732
62,WI,14-No Standing,1639
63,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


The code above uses [method chaining](https://tomaugspurger.net/posts/method-chaining/) to combine a series of operations into a single statement. You might find it useful to break the code up into multiple statements and inspect each of the intermediate results!

## Which vehicle body types are most frequently involved in parking violations?

We can also investigate which vehicle body types most commonly appear in parking violations

In [0]:
(
    df.groupby(["Vehicle Body Type"])
    .agg({"Summons Number": "count"})
    .rename(columns={"Summons Number": "Count"})
    .sort_values(["Count"], ascending=False)
)

,Count
Vehicle Body Type,
SUBN,6449007
4DSD,4402991
VAN,1317899
DELV,436430
PICK,429798
...,...
CARY,1
ISUZ,1
IXMR,1


## How do parking violations vary across days of the week?

In [0]:
weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

Out[11]: issue_weekday
Sunday        462992
Saturday     1108385
Monday       2488563
Wednesday    2760088
Tuesday      2809949
Friday       2891679
Thursday     2913951
Name: Summons Number, dtype: int64

It looks like there are fewer violations on weekends, which makes sense! During the week, more people are driving in New York City.

## Let's time it!

Loading and processing this data took a little time. Let's measure how long these pipelines take in Pandas:

In [0]:
%%time

df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=[
        "Registration State",
        "Violation Description",
        "Vehicle Body Type",
        "Issue Date",
        "Summons Number",
    ],
)

(
    df[["Registration State", "Violation Description"]]
    .value_counts()
    .groupby("Registration State")
    .head(1)
    .sort_index()
    .reset_index()
)

CPU times: user 10.9 s, sys: 3.3 s, total: 14.2 s
Wall time: 11.4 s


,Registration State,Violation Description,0
0,99,74-Missing Display Plate,835
1,AB,14-No Standing,22
2,AK,PHTO SCHOOL ZN SPEED VIOLATION,125
3,AL,PHTO SCHOOL ZN SPEED VIOLATION,3668
4,AR,PHTO SCHOOL ZN SPEED VIOLATION,537
...,...,...,...
60,VT,PHTO SCHOOL ZN SPEED VIOLATION,3024
61,WA,21-No Parking (street clean),3732
62,WI,14-No Standing,1639
63,WV,PHTO SCHOOL ZN SPEED VIOLATION,1185


In [0]:
%%time

(
    df.groupby(["Vehicle Body Type"])
    .agg({"Summons Number": "count"})
    .rename(columns={"Summons Number": "Count"})
    .sort_values(["Count"], ascending=False)
)

CPU times: user 2.41 s, sys: 173 ms, total: 2.58 s
Wall time: 2.57 s


,Count
Vehicle Body Type,
SUBN,6449007
4DSD,4402991
VAN,1317899
DELV,436430
PICK,429798
...,...
CARY,1
ISUZ,1
IXMR,1


In [0]:
%%time

weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

CPU times: user 5.53 s, sys: 653 ms, total: 6.18 s
Wall time: 6.18 s
Out[14]: issue_weekday
Sunday        462992
Saturday     1108385
Monday       2488563
Wednesday    2760088
Tuesday      2809949
Friday       2891679
Thursday     2913951
Name: Summons Number, dtype: int64

# Using cuDF's pandas accelerator mode (cudf.pandas)

Now, let's re-run the Pandas code above with the `cudf.pandas` extension loaded.

Typically, you should load the `cudf.pandas` extension as the first step in your notebook, before importing any modules. Here, we explicitly restart the kernel to simulate that behavior.

In [0]:
get_ipython().kernel.do_shutdown(restart=True)

Out[15]: {'status': 'ok', 'restart': True}

In [0]:
# %pip install  /dbfs/FileStore/cudf_private_cu11-23.10.0.231106-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --extra-index-url https://pypi.nvidia.com rmm-cu11

In [0]:
%pip install --upgrade pip plotly_express==0.4.1 nbformat

In [0]:
!pip list

In [0]:
!pip install --upgrade pip --extra-index-url=https://pypi.nvidia.com cudf-cu11 dask-cudf-cu11

In [0]:
%load_ext cudf.pandas

In [0]:
%%time

import pandas as pd

df = pd.read_parquet(
    "nyc_parking_violations_2022.parquet",
    columns=[
        "Registration State",
        "Violation Description",
        "Vehicle Body Type",
        "Issue Date",
        "Summons Number",
    ],
)

(
    df[["Registration State", "Violation Description"]]
    .value_counts()
    .groupby("Registration State")
    .head(1)
    .sort_index()
    .reset_index()
)

In [0]:
%%time

(
    df.groupby(["Vehicle Body Type"])
    .agg({"Summons Number": "count"})
    .rename(columns={"Summons Number": "Count"})
    .sort_values(["Count"], ascending=False)
)

In [0]:
%%time

weekday_names = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df["Issue Date"] = df["Issue Date"].astype("datetime64[ms]")
df["issue_weekday"] = df["Issue Date"].dt.weekday.map(weekday_names)

df.groupby(["issue_weekday"])["Summons Number"].count().sort_values()

Much faster! Operations that took 5-20 seconds can now potentially finish in just milliseconds without changing any code.

# Understanding Performance

cuDF's pandas accelerator mode provides profiling utilities to help you better understand performance. With these tools, you can identify which parts of your code ran on the GPU and which parts ran on the CPU.

They're accessible in the `cudf.pandas` namespace since the `cudf.pandas` extension was loaded above with `load_ext cudf.pandas`.

#### Colab Note
If you're running in Colab, the first time you run use the profiler it may take 10+ seconds due to Colab's debugger interacting with the built-in Python function [sys.settrace](https://docs.python.org/3/library/sys.html#sys.settrace) that we use for profiling. For demo purposes, this isn't an issue. Just run the cell again.

## Profiling Functionality

We can generate a per-function profile:

In [0]:
%%cudf.pandas.profile

small_df = pd.DataFrame({'a': [0, 1, 2], 'b': ["x", "y", "z"]})
small_df = pd.concat([small_df, small_df])

axis = 0
for i in range(0, 2):
    small_df.min(axis=axis)
    axis = 1

counts = small_df.groupby("a").b.count()

In [0]:
%%cudf.pandas.line_profile

small_df = pd.DataFrame({'a': [0, 1, 2], 'b': ["x", "y", "z"]})
small_df = pd.concat([small_df, small_df])

axis = 0
for i in range(0, 2):
    small_df.min(axis=axis)
    axis = 1

counts = small_df.groupby("a").b.count()

## Behind the scenes: What's going on here?

When you load cuDF's pandas accelerator mode, Pandas types like `Series` and `DataFrame` are replaced by proxy objects that dispatch operations to cuDF when possible. We can verify that `cudf.pandas` is active by looking at our `pd` variable:

In [0]:
pd

As a result, all pandas functions, methods, and created objects are proxies:

In [0]:
type(pd.read_csv)

Operations supported by cuDF will be **very** fast:

In [0]:
%%time
df.count(axis=0)

Operations not supported by cuDF will be slower, as they fall back to using Pandas (copying data between the CPU and GPU under the hood as needed). For example, cuDF does not currently support the `axis=` parameter to the `count` method. So this operation will run on the CPU and be noticeably slower than the previous one.

In [0]:
%%time
df.count(
    axis=1
)  # This will use pandas, because cuDF doesn't support axis=1 for the .count() method

But the story doesn't end here. We often need to mix our own code with third-party libraries that other people have written. Many of these libraries accept pandas objects as inputs.

# Using third-party libraries with cuDF's pandas accelerator mode

You can pass Pandas objects to third-party libraries when using `cudf.pandas`, just like you would when using regular Pandas.

Below, we show an example of using [plotly-express](https://plotly.com/python/plotly-express/) to visualize the data we've been processing:

## Visualizing which states have more pickup trucks relative to other vehicles?

In [0]:
import plotly.express as px

df = df.rename(
    columns={
        "Registration State": "reg_state",
        "Vehicle Body Type": "vehicle_type",
    }
)

# vehicle counts per state:
counts = df.groupby("reg_state").size().sort_index()
# vehicles with type "PICK" (Pickup Truck)
pickup_counts = df.where(df["vehicle_type"] == "PICK").groupby("reg_state").size()
# percentage of pickup trucks by state:
pickup_frac = ((pickup_counts / counts) * 100).rename("% Pickup Trucks")
del pickup_frac["MB"]  # (Manitoba is a huge outlier!)

# plot the results:
pickup_frac = pickup_frac.reset_index()
fig = px.choropleth(
    pickup_frac,
    locations="reg_state",
    color="% Pickup Trucks",
    locationmode="USA-states",
    scope="usa",
)

fig.show(renderer="databricks")

## Beyond just passing data: **Accelerating** third-party code

Being able to pass these proxy objects to libraries like Plotly is great, but the benefits don't end there.

When you enable cuDF's pandas accelerator mode, pandas operations running **inside the third-party library's functions** will also benefit from GPU acceleration where possible!

Below, you can see an image illustrating how `cudf.pandas` can accelerate the pandas backend in Ibis, a library that provides a unified DataFrame API to various backends. We ran this example on a system with an NVIDIA H100 GPU and an Intel Xeon Platinum 8480CL CPU.


By loading the `cudf.pandas` extension, pandas operations within Ibis can use the GPU with zero code change. It just works.

![ibis](https://drive.google.com/uc?id=1uOJq2JtbgVb7tb8qw8a2gG3JRBo72t_H)

# Conclusion

With cuDF's pandas accelerator mode, you can keep using pandas as your primary dataframe library. When things start to get a little slow, just load the `cudf.pandas` extension and run your existing code on a GPU!

To learn more about `cudf.pandas`, we encourage you to visit [rapids.ai/cudf-pandas](https://rapids.ai/cudf-pandas).